<a href="https://colab.research.google.com/github/hammadkhann/Towards-reducing-the-storage-cost-of-late-interation-reranker/blob/main/Neural_Reranking(RD3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pyterrier Setup**

In [1]:
!pip -q install python-terrier
import pyterrier as pt
pt.init()

     |████████████████████████████████| 95 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 255 kB 45.7 MB/s 
     |████████████████████████████████| 126 kB 37.2 MB/s 
     |████████████████████████████████| 596 kB 43.4 MB/s 
     |████████████████████████████████| 1.8 MB 34.3 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 6.3 MB 44.8 MB/s 
     |████████████████████████████████| 294 kB 33.9 MB/s 
     |████████████████████████████████| 291 kB 22.4 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


## **Importing useful libraries**



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
from google.colab import drive
from pyterrier.measures import *

pd.options.display.max_rows = None
pd.options.display.max_columns = None

## **Connect Google Drive**


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# **Preparing testing dataset**

## **DPH(Sparse Retrieval)**

## TREC-2019

In [4]:
br = pt.BatchRetrieve.from_dataset('msmarco_passage', 'terrier_stemmed_text', num_results=100, metadata = ['docno', 'text'], verbose=True)

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:273: UserWarning: Downloading index of > 2GB.
  warn("Downloading index of > 2GB.")


data.direct.bf:   0%|          | 0.00/486M [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/177M [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/377M [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/100M [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/67.5M [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/1.91G [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.33k [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/537 [00:00<?, ?iB/s]

22:46:51.349 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [5]:
dph_2019 = pd.DataFrame()
topics = pt.get_dataset('msmarco_passage').get_topics('test-2019')
topics = pt.get_dataset('msmarco_passage').get_qrels('test-2019')[["qid"]].drop_duplicates().merge(topics, on=['qid'])
for res in br.transform_gen(topics, batch_size=10):
  dph_2019 = dph_2019.append(res)
print("Number of topics: ", len(topics))

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

2019qrels-docs.txt:   0%|          | 0.00/183k [00:00<?, ?iB/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/3 [00:00<?, ?q/s]

Number of topics:  43


In [6]:
dph_2019.shape

(4205, 7)

## TREC-2020

In [7]:
dph_2020 = pd.DataFrame()
topics = pt.get_dataset('msmarco_passage').get_topics('test-2020')
topics = pt.get_dataset('msmarco_passage').get_qrels('test-2020')[["qid"]].drop_duplicates().merge(topics, on=['qid'])
for res in br.transform_gen(topics, batch_size=10):
  dph_2020 = dph_2020.append(res)
  
print("Number of topics: ", len(topics))

msmarco-test2020-queries.tsv.gz:   0%|          | 0.00/4.03k [00:00<?, ?iB/s]

2020qrels-docs.txt:   0%|          | 0.00/213k [00:00<?, ?iB/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/4 [00:00<?, ?q/s]

Number of topics:  54


In [8]:
dph_2020.shape

(5329, 7)

## **ColBERT(Dense Retrieval)**

## TREC-2019

In [9]:
index = pt.IndexFactory.of(pt.get_dataset("msmarco_passage").get_index("terrier_stemmed_text"))
print(index.getCollectionStatistics())

22:47:07.845 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [10]:
colbert_2019 = pt.io.read_results("/content/drive/MyDrive/E2E.2019.res")
# get document text for each docid
meta = index.getMetaIndex()
text = []
for index, row in colbert_2019.iterrows():
    text.append(meta.getItem('text', int(row['docno'])))
colbert_2019['text'] = text

# get query for each qid
topics_2019 = pt.get_dataset('msmarco_passage').get_topics('test-2019')
colbert_2019 = pd.merge(colbert_2019, topics_2019, on='qid', how='left')

# filter queries on 43 topics
colbert_2019 = colbert_2019[colbert_2019['qid'].isin(dph_2019['qid'].unique())]
# filter data on top100 results for each 43 topics
colbert_2019 = colbert_2019.groupby('qid').apply(lambda x : x.sort_values(by = 'score', ascending = False).head(100).reset_index(drop = True))
# reset index
colbert_2019.reset_index(drop=True, inplace=True)
colbert_2019.shape

(4300, 7)

## TREC-2020

In [11]:
colbert_2020 = pt.io.read_results("/content/drive/MyDrive/E2E.2020.res")
index = pt.IndexFactory.of(pt.get_dataset("msmarco_passage").get_index("terrier_stemmed_text"))

# get document text for each docid
meta = index.getMetaIndex()
text = []
for index, row in colbert_2020.iterrows():
    text.append(meta.getItem('text', int(row['docno'])))
colbert_2020['text'] = text

# get query for each qid
topics_2020 = pt.get_dataset('msmarco_passage').get_topics('test-2020')
colbert_2020 = pd.merge(colbert_2020, topics_2020, on='qid', how='left')

# filter queries on 43 topics
colbert_2020 = colbert_2020[colbert_2020['qid'].isin(dph_2020['qid'].unique())]
# filter data on top100 results for each 43 topics
colbert_2020 = colbert_2020.groupby('qid').apply(lambda x : x.sort_values(by = 'score', ascending = False).head(100).reset_index(drop = True))
# reset index
colbert_2020.reset_index(drop=True, inplace=True)
colbert_2020.shape

22:48:19.309 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


(5400, 7)

# **Pyterrier-ColBERT Setup**

In [12]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git

  Cloning https://github.com/terrierteam/pyterrier_colbert.git to /tmp/pip-req-build-h0i5dk5b
  Running command git clone -q https://github.com/terrierteam/pyterrier_colbert.git /tmp/pip-req-build-h0i5dk5b
  Cloning https://github.com/cmacdonald/ColBERT.git (to revision v0.2) to /tmp/pip-install-586ceje8/colbert_402010e1df9840d782d02d2238b610f8
  Running command git clone -q https://github.com/cmacdonald/ColBERT.git /tmp/pip-install-586ceje8/colbert_402010e1df9840d782d02d2238b610f8
  Running command git checkout -b v0.2 --track origin/v0.2
  Switched to a new branch 'v0.2'
  Branch 'v0.2' set up to track remote branch 'v0.2' from 'origin'.
     |████████████████████████████████| 769 kB 10.8 MB/s 
     |████████████████████████████████| 216 kB 40.3 MB/s 
     |████████████████████████████████| 15.5 MB 33.1 MB/s 
     |████████████████████████████████| 3.0 MB 32.7 MB/s 
     |████████████████████████████████| 895 kB 23.3 MB/s 
     |████████████████████████████████| 1.2 MB 36.6 MB/s 
   

In [13]:
from pyterrier_colbert.ranking import ColBERTFactory

## Updating pyterrier-ColBERT load methods to support latest version of transformers library.

In [14]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/Luyu/condenser

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 52 not upgraded.
Need to get 6,526 kB of archives.
After this operation, 14.7 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.0.2 [6,526 kB]
Fetched 6,526 kB in 1s (7,385 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/per

In [15]:
import os
import ujson
import torch
import random

from collections import defaultdict, OrderedDict

from colbert.parameters import DEVICE
import colbert.utils.utils
from colbert.utils.utils import print_message


def _load_checkpoint(path, model, optimizer=None, do_print=True):
    if do_print:
        print_message("#> Loading checkpoint", path)

    checkpoint = torch.load(path, map_location='cpu')

    state_dict = checkpoint['model_state_dict']
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k
        if k[:7] == 'module.':
            name = k[7:]
        new_state_dict[name] = v

    checkpoint['model_state_dict'] = new_state_dict

    model.load_state_dict(checkpoint['model_state_dict'], strict=False)

    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'], strict=False)

    if do_print:
        print_message("#> checkpoint['epoch'] =", checkpoint['epoch'])
        print_message("#> checkpoint['batch'] =", checkpoint['batch'])

    return checkpoint

colbert.utils.utils.load_checkpoint = _load_checkpoint
colbert.utils.utils.load_checkpoint.__globals__['load_checkpoint'] = _load_checkpoint
from colbert.modeling.colbert import ColBERT

def _load_model(args, do_print=True):
    colbert = ColBERT.from_pretrained('/content/condenser/',
                                      query_maxlen=args.query_maxlen,
                                      doc_maxlen=args.doc_maxlen,
                                      dim=args.dim,
                                      similarity_metric=args.similarity,
                                      mask_punctuation=args.mask_punctuation)
    colbert = colbert.to(DEVICE)

    print_message("#> Loading model checkpoint.", condition=do_print)

    checkpoint = load_checkpoint(args.checkpoint, colbert, do_print=do_print)

    colbert.eval()

    return colbert, checkpoint

In [16]:
import colbert.evaluation.load_model
import pyterrier_colbert.ranking
colbert.evaluation.load_model.load_model = _load_model
pyterrier_colbert.ranking.load_model = _load_model

# **Evaluation on TREC 2019**

## **Loading model checkpoints**

### Baseline

In [17]:
colbert_checkpoint="/content/drive/MyDrive/colbert-baseline-50000.dnn" 
colbert_factory = ColBERTFactory(colbert_checkpoint, None, None)
colbert_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  colbert_factory.text_scorer()
colbert_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  colbert_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 05, 22:50:50] #> Loading model checkpoint.
[Dec 05, 22:50:50] #> Loading checkpoint /content/drive/MyDrive/colbert-baseline-50000.dnn
[Dec 05, 22:51:09] #> checkpoint['epoch'] = 0
[Dec 05, 22:51:09] #> checkpoint['batch'] = 50000


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

### RD3: Multi-Task Learning (MTL)   

In [18]:
mtl_checkpoint="/content/drive/MyDrive/mtl_v2_cls+tok_losses-50000.dnn" 
mtl_factory = ColBERTFactory(mtl_checkpoint, None, None)
mtl_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  mtl_factory.text_scorer()
mtl_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  mtl_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 05, 22:51:16] #> Loading model checkpoint.
[Dec 05, 22:51:16] #> Loading checkpoint /content/drive/MyDrive/mtl_v2_cls+tok_losses-50000.dnn
[Dec 05, 22:51:37] #> checkpoint['epoch'] = 0
[Dec 05, 22:51:37] #> checkpoint['batch'] = 50000


## **Experiment**

### DPH(Sparse Retrieval) Re-ranking

In [ ]:
pt.Experiment(
    [colbert_pipeline, mtl_pipeline],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    filter_by_qrels=True,
    names=['DPH >> ColBERT', 'DPH >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,DPH >> ColBERT,0.668218,0.340573,0.832226
1,DPH >> ColBERT-mtl-objective,0.640291,0.335387,0.868605


### ColBERT(Dense Retrieval) Re-ranking

In [ ]:
pt.Experiment(
    [colbert_pipeline_ann, mtl_pipeline_ann],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT', 'ANN-ColBERT >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,ANN-ColBERT >> ColBERT,0.719955,0.396942,0.877907
1,ANN-ColBERT >> ColBERT-mtl-objective,0.640436,0.354623,0.862403


# **Evaluation on TREC 2020**

## **Loading model checkpoints**

### Baseline 

In [19]:
colbert_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  colbert_factory.text_scorer()
colbert_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  colbert_factory.text_scorer()

### RD3: Multi-Task Learning (MTL)   

In [20]:
mtl_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  mtl_factory.text_scorer()
mtl_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  mtl_factory.text_scorer()

## **Experiment**

### DPH(Sparse Retrieval) Re-ranking

In [ ]:
pt.Experiment(
    [colbert_pipeline_2020, mtl_pipeline_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['DPH >> ColBERT', 'DPH >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,DPH >> ColBERT,0.664682,0.395361,0.851235
1,DPH >> ColBERT-mtl-objective,0.659804,0.382391,0.847737


### ColBERT(Dense Retrieval) Re-ranking

In [22]:
pt.Experiment(
    [colbert_pipeline_ann_2020, mtl_pipeline_ann_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT', 'ANN-ColBERT >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,ANN-ColBERT >> ColBERT,0.690310,0.449931,0.844136
1,ANN-ColBERT >> ColBERT-mtl-objective,0.631162,0.396310,0.833848


# **CLS token scoring Comparison**

In [21]:
import torch
def slow_rerank_with_qembs(args, qembs, pids, passages, gpu=True):
    colbert = args.colbert
    inference = args.inference

    # make to 3d tensor
    Q = torch.unsqueeze(qembs, 0)
    if gpu:
        Q = Q.cuda()
    
    D_ = inference.docFromText(passages, bsize=args.bsize)
    if gpu:
      D_ = D_.cuda()

    scores = (Q[:, 0:1:, ] @ D_[:, 0:1:, ].permute(0, 2, 1)).max(2).values.sum(1)

    scores = scores.sort(descending=True)
    ranked = scores.indices.tolist()

    ranked_scores = scores.values.tolist()
    ranked_pids = [pids[position] for position in ranked]
    ranked_passages = [passages[position] for position in ranked]

    return list(zip(ranked_scores, ranked_pids, ranked_passages))

In [22]:
from pyterrier.transformer import TransformerBase
def text_scorer(self, query_encoded=False, doc_attr="text", verbose=False) -> TransformerBase:
    """
    Returns a transformer that uses ColBERT model to score the *text* of documents.
    """
    #input: qid, query, docno, text
    #OR
    #input: qid, query, query_embs, query_toks, query_weights, docno, text

    #output: qid, query, docno, score

    def _text_scorer(queries_and_docs):
        groupby = queries_and_docs.groupby("qid")
        rtr=[]
        with torch.no_grad():
            for qid, group in tqdm(groupby, total=len(groupby), unit="q") if verbose else groupby:
                query = group["query"].values[0]
                ranking = slow_rerank(self.args, query, group["docno"].values, group[doc_attr].values.tolist())
                for rank, (score, pid, passage) in enumerate(ranking):
                        rtr.append([qid, query, pid, score, rank])          
        return pd.DataFrame(rtr, columns=["qid", "query", "docno", "score", "rank"])

    # when query is encoded 
    def _text_scorer_qembs(queries_and_docs):
        groupby = queries_and_docs.groupby("qid")
        rtr=[]
        with torch.no_grad():
            for qid, group in tqdm(groupby, total=len(groupby), unit="q") if verbose else groupby:
                qembs = group["query_embs"].values[0]
                query = group["query"].values[0]
                ranking = slow_rerank_with_qembs(self.args, qembs, group["docno"].values, group[doc_attr].values.tolist())
                for rank, (score, pid, passage) in enumerate(ranking):
                        rtr.append([qid, query, pid, score, rank])          
        return pd.DataFrame(rtr, columns=["qid", "query", "docno", "score", "rank"])

    return pt.apply.generic(_text_scorer_qembs if query_encoded else _text_scorer)

In [23]:
import pyterrier_colbert.ranking
import pyterrier_colbert.pruning
colbert_factory.text_scorer = text_scorer
colbert_factory.text_scorer.__globals__['text_scorer'] = text_scorer
mtl_factory.text_scorer = text_scorer
mtl_factory.text_scorer.__globals__['text_scorer'] = text_scorer

In [24]:
mtl_cls_score_2019 = pt.transformer.SourceTransformer(colbert_2019) >> mtl_factory.query_encoder() >> mtl_factory.text_scorer(mtl_factory, query_encoded=True)
colbert_cls_score_2019 = pt.transformer.SourceTransformer(colbert_2019) >>  colbert_factory.query_encoder() >> colbert_factory.text_scorer(colbert_factory, query_encoded=True)

In [25]:
pt.Experiment(
    [colbert_cls_score_2019, mtl_cls_score_2019],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT_cls', 'ANN-ColBERT >> ColBERT_mtl_cls'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,ANN-ColBERT >> ColBERT_cls,0.485091,0.254003,0.694444
1,ANN-ColBERT >> ColBERT_mtl_cls,0.578223,0.302594,0.774382


In [26]:
mtl_cls_score_2020 = pt.transformer.SourceTransformer(colbert_2020) >> mtl_factory.query_encoder() >> mtl_factory.text_scorer(mtl_factory, query_encoded=True)
colbert_cls_score_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  colbert_factory.query_encoder() >> colbert_factory.text_scorer(colbert_factory, query_encoded=True)

In [31]:
pt.Experiment(
    [colbert_cls_score_2020, mtl_cls_score_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT_cls', 'ANN-ColBERT >> ColBERT_mtl_cls'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,ANN-ColBERT >> ColBERT_cls,0.435346,0.275212,0.574015
1,ANN-ColBERT >> ColBERT_mtl_cls,0.580869,0.365567,0.748148
